# Ergonomic Assessment API Test

This notebook demonstrates how to use the Ergonomic Assessment API to analyze images and videos for ergonomic risks.

In [1]:
import requests
import json
import time
import os
import glob
import matplotlib.pyplot as plt
from IPython.display import Image, display, HTML
import ipywidgets as widgets
from urllib.parse import urljoin

## Configure API URL

If the server is running on a different machine, you can update the base URL here.

In [2]:
# Base URL of the API
# Replace 'desktop-r8kamps' with your actual server hostname
BASE_URL = "http://desktop-r8kamps:5050/"  

# Add a function to let users easily set the server address
def set_server_address(ip_or_hostname, port=5050):
    global BASE_URL
    BASE_URL = f"http://{ip_or_hostname}:{port}/"
    print(f"Server address set to: {BASE_URL}")
    return BASE_URL

# Set server address to your hostname
set_server_address("desktop-r8kamps", 5050)

# Test the connection
try:
    # Just check if the server is running by pinging a simple endpoint
    response = requests.get(urljoin(BASE_URL, "predict/video/result?job_id=test"))
    print(f"Server response status: {response.status_code}")
    print(f"Response: {response.text}")
    
    if response.status_code in [200, 404]:  # 404 is expected for non-existent job
        print("✅ Server is running!")
    else:
        print("❌ Server responded with an unexpected status code")
except requests.exceptions.ConnectionError:
    print(f"❌ Failed to connect to server at {BASE_URL}")
    print("Make sure the server is running and the URL is correct.")

Server address set to: http://desktop-r8kamps:5050/
Server response status: 404
Response: {
  "error": "Job not found or expired"
}

✅ Server is running!


## Analyze an Image

This section demonstrates how to upload an image and get ergonomic analysis results.

In [3]:
# Find test images
test_dir = "/root/model/TA/Deploy/test"
if os.path.exists(test_dir):
    image_files = glob.glob(f"{test_dir}/*.jpg") + glob.glob(f"{test_dir}/*.jpeg") + glob.glob(f"{test_dir}/*.png")
    
    if image_files:
        print(f"Found {len(image_files)} images in test directory:")
        for img in image_files:
            print(f"  - {os.path.basename(img)}")
        
        # Use the first image for testing
        test_image_path = image_files[0]
        print(f"\nUsing {os.path.basename(test_image_path)} for testing")
    else:
        print("No image files found in test directory")
        test_image_path = None
else:
    print(f"Test directory not found: {test_dir}")
    test_image_path = None

Found 2 images in test directory:
  - 2B.jpg
  - 144714956.jpg

Using 2B.jpg for testing


In [4]:
def analyze_image(image_path):
    """Upload an image and get ergonomic analysis results"""
    print(f"Analyzing image: {image_path}")
    
    # Check if file exists
    if not os.path.exists(image_path):
        print(f"❌ File not found: {image_path}")
        return None
        
    # Prepare file for upload
    with open(image_path, 'rb') as f:
        files = {'image': (os.path.basename(image_path), f, 'image/jpeg')}
        
        # Make the request
        try:
            response = requests.post(urljoin(BASE_URL, "predict/image"), files=files)
            
            if response.status_code == 200:
                result = response.json().get('result')
                print(f"✅ Analysis complete!")
                return result
            else:
                print(f"❌ Error: {response.status_code}")
                try:
                    print(response.json())
                except:
                    print(response.text)
                return None
        except Exception as e:
            print(f"❌ Exception: {str(e)}")
            return None

In [5]:
# Process image if found
if test_image_path:
    # Analyze the image
    image_result = analyze_image(test_image_path)
    
    # Display the result
    if image_result:
        print(f"REBA Score: {image_result['reba_score']:.1f} ({image_result['risk_level']} Risk)")
        
        # Display component scores
        print("\nComponent Scores:")
        for component, score in image_result['component_scores'].items():
            print(f"  - {component.title()}: {score}")
            
        # Display visualization if available
        if 'visualization_path' in image_result:
            visualization_url = urljoin(BASE_URL, "output_images/" + image_result['visualization_path'])
            print(f"\nVisualization: {visualization_url}")
            display(Image(url=visualization_url))
            
        # Display feedback
        if 'feedback' in image_result:
            print("\nFeedback:")
            print(image_result['feedback'])
else:
    print("No test image available. Please add images to the test directory.")

Analyzing image: /root/model/TA/Deploy/test/2B.jpg
✅ Analysis complete!
REBA Score: 2.5 (Low Risk)

Component Scores:
  - Leg_Score: 2
  - Lower_Arm_Score: 2
  - Neck_Score: 1
  - Reba_Score: 2.5340089797973633
  - Trunk_Score: 2
  - Upper_Arm_Score: 1

Visualization: http://desktop-r8kamps:5050/output_images/2025-05-19/image_20250519151911_2_151913.png



Feedback:
Overall REBA Score: 2.5 - Low Risk. Action may be needed.

Focus on improving: Lower arm position, Leg posture (score 2).

Recommended actions:
- Position work to allow 90-110° elbow angles
- Ensure even weight distribution between legs
- Avoid prolonged static standing in awkward positions



## Analyze a Video

This section demonstrates how to upload a video for asynchronous processing and retrieve the results.

In [6]:
# Find test videos
test_dir = "/root/model/TA/Deploy/test"
if os.path.exists(test_dir):
    video_files = glob.glob(f"{test_dir}/*.mp4") + glob.glob(f"{test_dir}/*.avi") + glob.glob(f"{test_dir}/*.mov")
    
    if video_files:
        print(f"Found {len(video_files)} videos in test directory:")
        for vid in video_files:
            print(f"  - {os.path.basename(vid)}")
        
        # Use the first video for testing
        test_video_path = video_files[0]
        print(f"\nUsing {os.path.basename(test_video_path)} for testing")
    else:
        print("No video files found in test directory")
        test_video_path = None
else:
    print(f"Test directory not found: {test_dir}")
    test_video_path = None

Found 1 videos in test directory:
  - ergotest.mp4

Using ergotest.mp4 for testing


In [7]:
def wait_result(job_id, check_interval=15):
    """
    Wait for video analysis to complete, checking until job is done
    
    Args:
        job_id: The job ID to monitor
        check_interval: Time between status checks in seconds
        
    Returns:
        The analysis result when job is complete
    """
    import time
    
    print(f"Waiting for video analysis to complete (job ID: {job_id})...")
    
    start_time = time.time()
    last_progress = 0
    no_progress_count = 0
    
    while True:  # Continue checking until job is done
        # First check if result is available with helper function
        result = check_job_result(job_id)
        if result:
            return result
            
        # If no result yet, check status for progress updates
        try:
            status_response = requests.get(urljoin(BASE_URL, f"predict/video/status?job_id={job_id}"))
            
            if status_response.status_code == 200:
                status_data = status_response.json()
                
                # Update progress if available
                current_progress = status_data.get('progress', last_progress)
                
                # Check if progress is stalled
                if current_progress == last_progress:
                    no_progress_count += 1
                else:
                    no_progress_count = 0
                    # Silent progress update (no print)
                
                last_progress = current_progress
                
                # Check job status
                job_status = status_data.get('status', '')
                
                # Only consider job complete when status is 'done' (not just 100% progress)
                if job_status == 'done':
                    # Try to get results
                    result_response = requests.get(urljoin(BASE_URL, f"predict/video/result?job_id={job_id}"))
                    
                    if result_response.status_code == 200:
                        try:
                            result_data = result_response.json()
                            # Make sure result_data also has the 'done' status and result is populated
                            if result_data.get('status') == 'done' and 'result' in result_data and result_data['result'] is not None:
                                return result_data
                            # else continue waiting
                        except Exception as e:
                            pass  # Silent error handling
                    # else continue waiting
            # else continue waiting
                
        except Exception as e:
            pass  # Silent error handling
        
        # Wait before next check
        time.sleep(check_interval)
    
    # This should never be reached since we loop until job is done
    return None


def check_job_result(job_id):
    """Helper function to check if job is truly done with valid results"""
    try:
        response = requests.get(urljoin(BASE_URL, f"predict/video/result?job_id={job_id}"))
        if response.status_code == 200:
            result = response.json()
            if (result.get('status') == 'done' and 
                'result' in result and 
                result['result'] is not None):
                return result
    except Exception:
        pass  # Silent error handling
    return None


def display_results(result):
    """Display video analysis results in a simple way"""
    if not result or 'status' not in result or result['status'] != 'done' or 'result' not in result or result['result'] is None:
        print("❌ No valid results to display")
        return
    
    video_result = result['result']
    
    print("\n===== VIDEO ANALYSIS RESULTS =====")
    print(f"REBA Score: {video_result.get('avg_reba_score', 'N/A'):.2f}")
    print(f"Risk Level: {video_result.get('risk_level', 'N/A')}")
    
    # Component scores
    if 'avg_component_scores' in video_result:
        print("\nComponent Scores:")
        for component, score in video_result['avg_component_scores'].items():
            print(f"  - {component.title()}: {score:.2f}")
    
    # Recommendations
    if 'recommendations' in video_result and video_result['recommendations']:
        print("\nRecommendations:")
        for i, rec in enumerate(video_result['recommendations'], 1):
            print(f"  {i}. {rec}")
    
    # High risk periods
    if 'high_risk_periods' in video_result and video_result['high_risk_periods']:
        print("\nHigh Risk Periods:")
        for i, period in enumerate(video_result['high_risk_periods'], 1):
            print(f"  {i}. {period.get('start', 0):.1f}s - {period.get('end', 0):.1f}s ({period.get('average_reba', 0):.2f} REBA)")
    
    # Feedback
    if 'feedback' in video_result:
        print("\nFeedback:")
        print(video_result['feedback'])
    
    print("====================================")

def display_segmented_results(result):
    """Display video analysis results for segmented video"""
    if not result or 'status' not in result or result['status'] != 'done' or 'result' not in result or result['result'] is None:
        print("❌ No valid results to display")
        return
    
    video_result = result['result']
    
    # Check if this is a segmented video result
    if 'segments' not in video_result and 'video_metadata' not in video_result:
        print("This appears to be a single-segment result. Using standard display...")
        return display_results(result)
    
    # Display metadata
    print("\n===== SEGMENTED VIDEO ANALYSIS RESULTS =====")
    metadata = video_result.get('video_metadata', {})
    print(f"Filename: {metadata.get('filename', 'Unknown')}")
    print(f"Duration: {metadata.get('duration_seconds', 0):.1f} seconds")
    print(f"Segments: {metadata.get('segments_count', 0)} (each {metadata.get('segment_duration_minutes', 0)} minutes)")
    
    # Display overall scores if available
    if 'overall_avg_reba_score' in video_result:
        print(f"\nOverall REBA Score: {video_result['overall_avg_reba_score']:.2f}")
        print(f"Overall Risk Level: {video_result.get('overall_risk_level', 'Unknown')}")
    
    # Display highest risk segment
    if 'highest_risk_segment' in video_result:
        high_risk = video_result['highest_risk_segment']
        segment_time = high_risk.get('segment_time', {})
        start_time = segment_time.get('start_time', 0)
        end_time = segment_time.get('end_time', 0)
        print(f"\nHighest Risk Segment: {high_risk.get('segment_index', 0) + 1}")
        print(f"  Time: {start_time:.1f}s - {end_time:.1f}s")
        print(f"  REBA Score: {high_risk.get('segment_reba_score', 0):.2f}")
    
    # Display overall recommendations
    if 'overall_recommendations' in video_result and video_result['overall_recommendations']:
        print("\nOverall Recommendations:")
        for i, rec in enumerate(video_result['overall_recommendations'], 1):
            print(f"  {i}. {rec}")
    
    # Display segment details
    segments = video_result.get('segments', [])
    print(f"\nSegment Details ({len(segments)} segments):")
    
    for i, segment in enumerate(segments, 1):
        segment_info = segment.get('segment_info', {})
        start_time = segment_info.get('start_time', 0)
        end_time = segment_info.get('end_time', 0)
        
        print(f"\nSegment {i}: {start_time:.1f}s - {end_time:.1f}s")
        print(f"  REBA Score: {segment.get('avg_reba_score', 0):.2f} ({segment.get('risk_level', 'Unknown')} Risk)")
        print(f"  Processed Frames: {segment.get('processed_frames', 0)}")
        
        # Component scores for this segment
        if 'avg_component_scores' in segment:
            print("  Component Scores:")
            for component, score in segment['avg_component_scores'].items():
                print(f"    - {component.title()}: {score:.2f}")
    
    print("\n===================================================")


def test_video(video_path=None, use_segmentation=None, segment_minutes=5):
    """
    Test function for video processing with optional segmentation
    
    Args:
        video_path: Path to the video file, or None to find videos in the test directory
        use_segmentation: Whether to use segmentation (None to prompt user, True or False to set)
        segment_minutes: Duration of each segment in minutes (if segmentation is enabled)
        
    Returns:
        The analysis result
    """
    import os
    import glob
    
    # If no path provided, try to find videos in the test directory
    if video_path is None:
        test_dir = "/root/model/TA/Deploy/test"
        if not os.path.exists(test_dir):
            print(f"❌ Test directory not found: {test_dir}")
            return
            
        video_files = glob.glob(f"{test_dir}/*.mp4") + glob.glob(f"{test_dir}/*.avi") + glob.glob(f"{test_dir}/*.mov")
        
        if not video_files:
            print(f"❌ No video files found in test directory: {test_dir}")
            return
            
        video_path = video_files[0]
        print(f"Found {len(video_files)} videos in test directory:")
        for vid in video_files:
            print(f"  - {os.path.basename(vid)}")
        print(f"\nUsing {os.path.basename(video_path)} for testing")
    
    if not os.path.exists(video_path):
        print(f"❌ Video file not found: {video_path}")
        return
    
    print(f"Testing video processing with: {os.path.basename(video_path)}")
    
    # Ask user about segmentation if not specified
    if use_segmentation is None:
        while True:
            choice = input(f"Use segmentation? (y/n) [Default segment size: {segment_minutes} minutes]: ").lower().strip()
            if choice in ('y', 'yes'):
                use_segmentation = True
                try:
                    new_segment_minutes = input(f"Enter segment duration in minutes (press Enter for default: {segment_minutes}): ").strip()
                    if new_segment_minutes:
                        segment_minutes = int(new_segment_minutes)
                except:
                    print(f"Invalid input. Using default: {segment_minutes} minutes")
                break
            elif choice in ('n', 'no'):
                use_segmentation = False
                break
            else:
                print("Please enter 'y' or 'n'")
    
    # 1. Submit video for processing
    print("Step 1: Submitting video for processing...")
    with open(video_path, 'rb') as f:
        files = {'video': (os.path.basename(video_path), f, 'video/mp4')}
        
        # Add segmentation parameter if enabled
        data = {}
        if use_segmentation:
            data['segment_duration_minutes'] = segment_minutes
            print(f"Segmentation enabled: {segment_minutes} minute segments")
        else:
            print("Segmentation disabled: processing as a single segment")
            
        response = requests.post(urljoin(BASE_URL, "predict/video"), files=files, data=data)
    
    if response.status_code != 200:
        print(f"❌ Failed to submit video: {response.status_code}")
        try:
            print(response.json())
        except:
            print(response.text)
        return
    
    job_id = response.json().get('job_id')
    if not job_id:
        print("❌ No job ID returned")
        return
    
    print(f"✅ Video submitted successfully! Job ID: {job_id}")
    
    # 2. Wait for processing to complete
    print("\nStep 2: Waiting for processing to complete...")
    result = wait_result(job_id)
    
    if not result:
        print("\n❌ Failed to get results")
        return
    
    # 3. Display results
    print("\nStep 3: Results ready. Displaying analysis:")
    
    # Check if this is a segmented result
    if ('result' in result and 
        isinstance(result['result'], dict) and 
        ('segments' in result['result'] or 'video_metadata' in result['result'])):
        display_segmented_results(result)
    else:
        display_results(result)
    
    return result


# Add a function to check a specific job ID
def check_specific_job(job_id, display=True):
    """Check the status and results of a specific job ID"""
    print(f"Checking job ID: {job_id}")
    
    # First check status
    try:
        status_response = requests.get(urljoin(BASE_URL, f"predict/video/status?job_id={job_id}"))
        if status_response.status_code == 200:
            status_data = status_response.json()
            print(f"Job status: {status_data.get('status')}")
            print(f"Progress: {status_data.get('progress', 0)}%")
        else:
            print(f"Failed to get status: {status_response.status_code}")
    except Exception as e:
        print(f"Error checking status: {str(e)}")
    
    # Then check result
    try:
        result_response = requests.get(urljoin(BASE_URL, f"predict/video/result?job_id={job_id}"))
        if result_response.status_code == 200:
            result_data = result_response.json()
            print(f"Result status: {result_data.get('status')}")
            if 'result' in result_data:
                if result_data['result'] is None:
                    print("Result field exists but is None")
                else:
                    print("Result exists!")
                    if display:
                        print("Displaying result data:")
                        # Check if this is a segmented result
                        if ('result' in result_data and 
                            isinstance(result_data['result'], dict) and 
                            ('segments' in result_data['result'] or 'video_metadata' in result_data['result'])):
                            display_segmented_results(result_data)
                        else:
                            display_results(result_data)
                    return result_data
            else:
                print("No 'result' field in response")
        else:
            print(f"Failed to get result: {result_response.status_code}")
    except Exception as e:
        print(f"Error checking result: {str(e)}")
    
    return None

In [8]:
test_video()

Found 1 videos in test directory:
  - ergotest.mp4

Using ergotest.mp4 for testing
Testing video processing with: ergotest.mp4


Use segmentation? (y/n) [Default segment size: 5 minutes]:  y
Enter segment duration in minutes (press Enter for default: 5):  1


Step 1: Submitting video for processing...
Segmentation enabled: 1 minute segments
✅ Video submitted successfully! Job ID: 97816574-06c5-4bea-a08d-f1b6dde6d04f

Step 2: Waiting for processing to complete...
Waiting for video analysis to complete (job ID: 97816574-06c5-4bea-a08d-f1b6dde6d04f)...

Step 3: Results ready. Displaying analysis:

===== SEGMENTED VIDEO ANALYSIS RESULTS =====
Filename: video.mp4
Duration: 179.3 seconds
Segments: 3 (each 1 minutes)

Overall REBA Score: 3.36
Overall Risk Level: Medium

Highest Risk Segment: 1
  Time: 0.0s - 60.0s
  REBA Score: 3.91

Overall Recommendations:
  1. Consider ergonomic adjustments to your workstation.
  2. Continue maintaining good posture with minor adjustments.
  3. Take regular breaks to change posture and reduce strain.
  4. Ensure even weight distribution between both legs.
  5. Use an anti-fatigue mat if standing for long periods.

Segment Details (3 segments):

Segment 1: 0.0s - 60.0s
  REBA Score: 3.91 (Medium Risk)
  Processe

{'created_at': 1747642784.5931997,
 'expire_at': 1747729423.5053449,
 'finished_at': 1747643023.505344,
 'result': {'highest_risk_segment': {'segment_index': 0,
   'segment_reba_score': 3.9115612441769234,
   'segment_time': {'end_frame': 1500,
    'end_time': 60.0,
    'segment_index': 0,
    'start_frame': 0,
    'start_time': 0.0}},
  'overall_avg_reba_score': 3.3569917401877,
  'overall_recommendations': ['Consider ergonomic adjustments to your workstation.',
   'Continue maintaining good posture with minor adjustments.',
   'Take regular breaks to change posture and reduce strain.',
   'Ensure even weight distribution between both legs.',
   'Use an anti-fatigue mat if standing for long periods.'],
  'overall_risk_level': 'Medium',
  'segments': [{'action_level': 2,
    'action_text': 'Action necessary',
    'angle_statistics': {'left_leg': {'avg': 99.74828315368632,
      'max': 100.0,
      'min': 90.48159488175138,
      'std': 1.309640256328053},
     'left_lower_arm': {'avg':

## Using a Previous Job ID

If you've already submitted a video and want to check its results:

In [7]:
def get_video_result(job_id):
    """Get the final result of a video analysis job"""
    try:
        response = requests.get(urljoin(BASE_URL, f"predict/video/result?job_id={job_id}"))
        if response.status_code == 200:
            return response.json()
        else:
            print(f"❌ Error: {response.status_code}")
            try:
                print(response.json())
            except:
                print(response.text)
            return None
    except Exception as e:
        print(f"❌ Exception: {str(e)}")
        return None
        
def deep_debug_result(job_id):
    """More thorough debugging for video results"""
    print(f"\n===== DEEP DEBUGGING FOR JOB {job_id} =====")
    
    # Get the job result
    response = requests.get(urljoin(BASE_URL, f"predict/video/result?job_id={job_id}"))
    
    if response.status_code != 200:
        print(f"Error: Got status code {response.status_code}")
        print(f"Response: {response.text}")
        return
    
    try:
        # Parse the JSON response
        job_data = response.json()
        print(f"Job status: {job_data.get('status')}")
        
        # Check for result key
        if 'result' not in job_data:
            print("Error: No 'result' key found in job data")
            print(f"Job data keys: {list(job_data.keys())}")
            print(f"Job data content: {json.dumps(job_data, indent=2)[:1000]}")
            return
        
        # Analyze the result structure
        result = job_data['result']
        if result is None:
            print("Error: Result is None")
            return
            
        print(f"Result type: {type(result)}")
        
        if isinstance(result, dict):
            print(f"Result keys: {list(result.keys())}")
            
            # Check for segments
            if 'segments' in result:
                print(f"Segmented video detected with {len(result['segments'])} segments")
                
                # Check first segment
                if result['segments']:
                    first_segment = result['segments'][0]
                    print(f"First segment keys: {list(first_segment.keys())}")
                    
                    # Check for key fields
                    if 'avg_reba_score' in first_segment:
                        print(f"First segment REBA score: {first_segment['avg_reba_score']}")
                    
                    if 'avg_component_scores' in first_segment:
                        print(f"First segment component scores available")
            
            # Check for single segment data
            elif 'avg_reba_score' in result:
                print(f"Single segment video with REBA score: {result['avg_reba_score']}")
                
                if 'avg_component_scores' in result:
                    print(f"Component scores: {result['avg_component_scores']}")
            
            else:
                print("Error: Neither segmented nor single-segment structure detected")
                print(f"First 1000 chars of result: {json.dumps(result, indent=2)[:1000]}")
        else:
            print(f"Error: Result is not a dictionary")
            print(f"Result value: {result}")
    
    except json.JSONDecodeError:
        print(f"Error: Failed to parse JSON response: {response.text[:500]}")
    except Exception as e:
        print(f"Error during debugging: {str(e)}")
    
    print("===== END DEEP DEBUGGING =====\n")

# Test function with your stored job ID
def test_with_specific_job_id(job_id):
    """Test function to analyze a specific job ID"""
    print(f"Analyzing job ID: {job_id}")
    
    # First do deep debugging
    deep_debug_result(job_id)
    
    # Try to fix and display results
    result = get_video_result(job_id)
    
    if result and result.get('status') == 'done':
        print("\nSuccessfully retrieved result data")
        
        # Try to extract video result from different possible structures
        video_result = None
        
        if 'result' in result:
            raw_result = result['result']
            
            # Case 1: Result is directly the data we need
            if isinstance(raw_result, dict) and 'avg_reba_score' in raw_result:
                video_result = raw_result
                print("Found result data directly in the 'result' field")
                
            # Case 2: Result contains segments
            elif isinstance(raw_result, dict) and 'segments' in raw_result:
                if raw_result['segments'] and isinstance(raw_result['segments'][0], dict):
                    # Use the first segment as the result
                    video_result = raw_result
                    print(f"Found segmented result with {len(raw_result['segments'])} segments")
                
            # Case 3: Result is in an unexpected format
            elif raw_result is not None:
                print(f"Result is in unexpected format: {type(raw_result)}")
                if isinstance(raw_result, dict):
                    print(f"Result keys: {list(raw_result.keys())}")
        else:
            print("No 'result' key found in response")
        
        # Display results if found
        if video_result:
            if 'segments' in video_result:
                # For segmented videos
                segments = video_result['segments']
                print(f"\nSegmented Video Results ({len(segments)} segments):")
                
                if 'overall_avg_reba_score' in video_result:
                    print(f"Overall REBA Score: {video_result['overall_avg_reba_score']}")
                    print(f"Risk Level: {video_result['overall_risk_level']}")
                
                # Show segment summaries
                for i, segment in enumerate(segments):
                    start_time = segment['segment_info']['start_time']
                    end_time = segment['segment_info']['end_time']
                    print(f"\nSegment {i+1}: {start_time:.1f}s - {end_time:.1f}s")
                    print(f"REBA Score: {segment.get('avg_reba_score')} ({segment.get('risk_level')})")
            else:
                # For single segment
                print(f"\nVideo Analysis Results:")
                print(f"REBA Score: {video_result.get('avg_reba_score')}")
                print(f"Risk Level: {video_result.get('risk_level')}")
        else:
            print("\nFailed to extract results from the response")
    else:
        print("\nFailed to retrieve results")

# Let's test with a recent job ID 
# You can replace this with one from your logs
job_id_to_test = "5b78abb7-e57e-4aa9-8502-8f2b6aa3a43d"
test_with_specific_job_id(job_id_to_test)

Analyzing job ID: 289db0fd-7a89-45cf-ad75-23228e361e2f

===== DEEP DEBUGGING FOR JOB 289db0fd-7a89-45cf-ad75-23228e361e2f =====
Job status: done
Result type: <class 'dict'>
Result keys: ['action_level', 'action_text', 'angle_statistics', 'avg_component_scores', 'avg_reba_score', 'feedback', 'high_risk_periods', 'high_risk_periods_count', 'processed_frames', 'reba_statistics', 'recommendations', 'risk_level', 'segment_info']
Single segment video with REBA score: 3.352310918509688
Component scores: {'leg': 2.0239726027397262, 'lower_arm': 1.832191780821918, 'neck': 1.2842465753424657, 'trunk': 2.3732876712328768, 'upper_arm': 1.3470319634703196}
===== END DEEP DEBUGGING =====


Successfully retrieved result data
Found result data directly in the 'result' field

Video Analysis Results:
REBA Score: 3.352310918509688
Risk Level: Medium
